In [1]:
import boto3
import json

In [2]:
region = 'us-east-1' #'ENTER YOUR REGION HERE'
TEXT_GENERATION_MODEL_ENDPOINT_NAME = 'jumpstart-dft-hf-llm-falcon-7b-instruct-bf16' #'ENTER YOUR TEXT GENERATION MODEL END POINT NAME HERE'
kendra_index_id = 'b52d1029-70a1-4e81-8cd7-2082e5dc0e9b'  #'ENTER YOUR KENDRA INDEX ID HERE'

In [3]:
sagemaker_runtime_client = boto3.client('runtime.sagemaker',
                            region_name=region)

kendra_client = boto3.client('kendra',
                            region_name=region)

bedrock_runtime = boto3.client('bedrock-runtime',
                            region_name=region)

In [4]:
def get_kendra_results(query):
    result = ''
    context = []

    response = kendra_client.query(QueryText= query, IndexId=kendra_index_id)

    first_result_type = ''
    if response['TotalNumberOfResults']!=0:
        first_result_type = response['ResultItems'][0]['Type']
        
    if first_result_type == 'QUESTION_ANSWER' or first_result_type == 'ANSWER':
        result = response['ResultItems'][0]['DocumentExcerpt']['Text']
    
    elif first_result_type == 'DOCUMENT':
        for query_result in response["ResultItems"]:
            if query_result["Type"]=="DOCUMENT":
                answer_text = query_result["DocumentExcerpt"]["Text"]
                context.append(answer_text)
            
            for text in context:
                result += text
    
    return result

In [5]:
query = "What are the 5 pillars of well architected framework?"
context = get_kendra_results(query)
context


'The AWS Well-Architected Framework is based on five pillars — operational\nexcellence, security, reliability, performance efficiency, and cost optimization.\n\n\nTable\xa01.\xa0The pillars of the AWS Well-Architected Framework\n\n\nName Description\nOperational Excellence The ability to run and monitor systems to'

In [6]:
SM_JS_MODEL = False
BEDROCK_MODEL = True

In [20]:

if SM_JS_MODEL == True:
        template = """
        You are a helpful, polite, fact-based agent.
        If you don't know the answer, just say that you don't know.
        Please answer the following question using the context provided. 

        CONTEXT: 
        {context}
        =========
        QUESTION: {question} 
        ANSWER: """

if BEDROCK_MODEL == True:
        template = f"""Human: You are a helpful, polite, fact-based agent.
        If you don't know the answer, just say that you don't know.
        Please answer the following question using the context provided. 

        CONTEXT: 
        {context}
        =========
        QUESTION: {query} 
        
        
        Assistant: """


In [21]:
prompt = template.format(context=context, question=query)
print(prompt)

Human: You are a helpful, polite, fact-based agent.
        If you don't know the answer, just say that you don't know.
        Please answer the following question using the context provided. 

        CONTEXT: 
        The AWS Well-Architected Framework is based on five pillars — operational
excellence, security, reliability, performance efficiency, and cost optimization.


Table 1. The pillars of the AWS Well-Architected Framework


Name Description
Operational Excellence The ability to run and monitor systems to
        QUESTION: What are the 5 pillars of well architected framework? 
        
        
        Assistant: 


In [9]:
if SM_JS_MODEL == True:
    response_model = sagemaker_runtime_client.invoke_endpoint(
        EndpointName=TEXT_GENERATION_MODEL_ENDPOINT_NAME,
        Body=json.dumps(
            {"inputs": prompt, "parameters": {"max_new_tokens": 500}}
        ),
        ContentType="application/json",
    )
    response = json.loads(response_model["Body"].read())

    print(response[0]["generated_text"])

In [23]:
if BEDROCK_MODEL == True:
    anthropic_model = 'anthropic.claude-v1'
    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 250})
    
    accept = 'application/json'
    contentType = 'application/json'

    response = bedrock_runtime.invoke_model(body=body, modelId=anthropic_model, 
                                    accept=accept, 
                                    contentType=contentType)
    
    response_body = json.loads(response.get('body').read())
    
    print(response_body['completion'])

 The 5 pillars of the AWS Well-Architected Framework are:

1. Operational Excellence 
2. Security
3. Reliability
4. Performance Efficiency
5. Cost Optimization
